## Parameters that can be changed

In [38]:
raw_data_file_name = "Sales_History_Apodaca.csv"
raw_data_file_sheet_name = 'Sales_History_Apodaca'
filter_date = '2023-02-11'

logistic_data_file_name = 'US CAN ZIP CODE FILE 1.xlsx'
logistic_data_sheet_name = 'us can zip'

FG_health_file_name = 'FG_Health_Apodaca.csv'

# Coordinates of Warehouses PHR --> McAllen, Texas; FDW --> Florence, Kentucky; IDW --> Indianapolis, AW --> Ontario, California
Warehouse_Coordinate = {
    'PHR': (26.1570071, -98.321983),
    'FDW': (38.9860247, -84.6139836),
    'IDW': (39.7332942, -86.3482675),
    'AW': (34.0434872, -117.5760558)
}

warehouse_lead_time ={
    'PHR' : 3,
    'FDW' : 7,
    'IDW' : 7,
    'AW' : 7
}

warehouse_name = ['PHR', 'FDW', 'IDW', 'AW']

warehouse_to_consider = ['FDW', 'IDW', 'PHR', 'AW']

current_date = '2025-02-17'

In [39]:
# Importing the required packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from geopy.distance import geodesic
from scipy.stats import norm

%matplotlib inline

In [40]:
# Reading the CSV file
dtype_column = {'Material Number': str,
                'Ship-To Postal Code': str}
df = pd.read_csv(raw_data_file_name, dtype=dtype_column)

In [41]:
df['Billing Date'] = pd.to_datetime(df['Billing Date'])
df['Billing Quantity in Base Unit'] = pd.to_numeric(df['Billing Quantity in Base Unit'], errors='coerce')

In [42]:
# Removing the unwanted columns that are not named
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.iloc[:-3]

In [43]:
print(f"Number of Unique postal codes: {df['Ship-To Postal Code'].nunique()}")

Number of Unique postal codes: 4212


In [44]:
# Extracting only the first 5 characters in the 'Ship to Postal Code' column
df['Ship-To Postal Code'] = df['Ship-To Postal Code'].str[:5]

## Reading complete sales data

In [45]:
# Reading the CSV file
dtype_column_1 = {'Material Number': str,
                'Ship-To Postal Code': str
                 }

# Define a converter function to remove commas and handle empty strings
def remove_commas(x):
    if x == '':
        return np.nan
    return float(x.replace(',', ''))

# Specify the converters for columns that need to be numeric
converters = {
    'Billing Quantity in Base Unit': remove_commas
}

# Read the CSV file with specified data types and converters
df1 = pd.read_csv("Sales_History_Apodaca_All.csv", dtype=dtype_column_1, converters=converters)

df1 = df1.iloc[:-3]
df1 = df1.reset_index(drop=True)

In [46]:
df = df.dropna(subset=['Ship-To Postal Code'])
df = df.reset_index(drop=True)

In [47]:
df_filter = df[['Material Number', 
                'Ship-To Postal Code', 
                'Ship-To Customer City', 
                'Ship-To Region', 
                'Billing Quantity in Base Unit', 
                'Shipping Plant']]
df_filter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115005 entries, 0 to 115004
Data columns (total 6 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Material Number                115005 non-null  object 
 1   Ship-To Postal Code            115005 non-null  object 
 2   Ship-To Customer City          115005 non-null  object 
 3   Ship-To Region                 115005 non-null  object 
 4   Billing Quantity in Base Unit  114895 non-null  float64
 5   Shipping Plant                 115005 non-null  object 
dtypes: float64(1), object(5)
memory usage: 5.3+ MB


In [48]:
df_filter['Billing Quantity in Base Unit'] = pd.to_numeric(df_filter['Billing Quantity in Base Unit'], errors='coerce')

C:\Users\610168286\AppData\Local\Temp\ipykernel_21056\1888360844.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter['Billing Quantity in Base Unit'] = pd.to_numeric(df_filter['Billing Quantity in Base Unit'], errors='coerce')


In [49]:
df_group = df_filter.groupby(['Material Number', 
                              'Ship-To Postal Code', 
                              'Ship-To Customer City', 
                              'Ship-To Region',
                              'Shipping Plant']).agg({'Billing Quantity in Base Unit': 'sum'}).reset_index()

### Loading the Logsitic Zip code data

In [50]:
# Reading the Zip code data that has Lat & long details based on Zip code
df_logistic = pd.read_excel(logistic_data_file_name, sheet_name= logistic_data_sheet_name)

In [51]:
# Filtering only US Zip code data
df_logistic.drop(df_logistic[df_logistic['COUNTRY'] != 'US'].index, inplace =True)

### Merging Sales data & Logistics data

In [52]:
# Merging the two DataFrames using left join
df_group = pd.merge(df_group, df_logistic[['zip'] + ['LAT', 'LON']],
             left_on = 'Ship-To Postal Code',
             right_on = 'zip',
             how = 'left')


df_group.drop(columns= ['zip'], inplace = True)

## Distance Calculation

In [53]:
# Function to calculate the distance

def calculate_distance(row, j):
    if pd.notnull(row['LAT']) and pd.notnull(row['LON']):
        return geodesic((row['LAT'], row['LON']), j).miles
    else:
        return np.nan

for i, j in Warehouse_Coordinate.items():
    if i in warehouse_to_consider:
        df_group[i] = df_group.apply(lambda row: calculate_distance(row, j), axis = 1)

In [54]:
df_group['Nearest Warehouse'] = df_group[warehouse_to_consider].idxmin(axis = 1)

C:\Users\610168286\AppData\Local\Temp\ipykernel_21056\3713160561.py:1: FutureWarning: The behavior of DataFrame.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  df_group['Nearest Warehouse'] = df_group[warehouse_to_consider].idxmin(axis = 1)


### FG Health Data

In [55]:
dtype_column = {'Material Number': str,
                'SS > 0': str}
df_FG = pd.read_csv(FG_health_file_name, dtype=dtype_column)

C:\Users\610168286\AppData\Local\Temp\ipykernel_21056\1111916359.py:3: DtypeWarning: Columns (19,20,21,65,69,70,90,111,112,148) have mixed types. Specify dtype option on import or set low_memory=False.
  df_FG = pd.read_csv(FG_health_file_name, dtype=dtype_column)


### Merging FG Health

In [56]:
# Merging FG health to filter out part numbers without safety stock
df_group = pd.merge(df_group, df_FG[['Material Number', 'SS > 0']],
                   on = 'Material Number',
                   how = 'left')
df_group = df_group.reset_index(drop=True)

In [57]:
df_group = df_group[df_group['SS > 0'] != 'N']

## Customer Concentration Calculation

In [58]:
def customer_concentration(df, df_name):
    # Grouping data to find the total billing quantity for each part number based on nearest warehouse
    
    part_warehouse_demand = df.groupby(['Material Number', 'Nearest Warehouse'])['Billing Quantity in Base Unit'].sum().reset_index()
    part_warehouse_demand.rename(columns={'Billing Quantity in Base Unit' : 'Demand Per Warehouse'}, inplace = True)
    
    # Grouping data to find the total billing quantity for each part number
    total_part_demand = df.groupby('Material Number')['Billing Quantity in Base Unit'].sum().reset_index()
    total_part_demand.rename(columns={'Billing Quantity in Base Unit' : 'Total Demand'}, inplace = True)

    df_merge = pd.merge(part_warehouse_demand, total_part_demand,
                   on = 'Material Number')
    '''
    # Converting the datatype to numeric
    df_merge['Demand Per Warehouse'] = pd.to_numeric(df_merge['Demand Per Warehouse'], errors='coerce')
    df_merge['Total Demand'] = pd.to_numeric(df_merge['Total Demand'], errors='coerce')
    '''
    

    df_merge['% Concentration'] = ((df_merge['Demand Per Warehouse']/df_merge['Total Demand']) * 100).round()

    # Dynamically name the resulting DataFrame
    globals()[df_name] = df_merge
    
    return df_merge


In [59]:
df_group = customer_concentration(df_group, 'df_group_merge')

## Pivoting the Dataframe

In [60]:
def pivot_dataframe(df):
    # Pivoting the df_merge DataFrame to find the concentration of parts demand on warehouse
    df_pivot = df.pivot_table(index = ['Material Number', 'Total Demand'],
                                   columns = 'Nearest Warehouse',
                                   values = '% Concentration').reset_index()
    df_pivot = df_pivot.fillna(0)
    df_pivot.columns.name = None
    df_pivot.reset_index(drop = True, inplace = True)

    # Dynamically name the resulting DataFrame
    #globals()[df_name] = df_pivot

    rename_dict = {warehouse: f'% {warehouse}' for warehouse in warehouse_to_consider}
    df_pivot.rename(columns=rename_dict, inplace = True)
    
    return df_pivot

In [61]:
df_group_pivot = pivot_dataframe(df_group_merge)

df_group_concentration = df_group_pivot.copy()

In [62]:
# This lead time is calculated based on source location to warehouse

for warehouse in warehouse_to_consider:
    df_group_concentration[f'{warehouse} Lead Time'] = warehouse_lead_time[warehouse]

## Creating new DataFrame to calculate Standard deviation & Average usage

In [63]:
# Creating this DataFrame to have a column with both part number and date, so as to merge later

df_std_copy = df1.copy()
df_std_copy = df_std_copy[['Material Number', 'Billing Quantity in Base Unit' , 'Billing Date']]
df_std_copy['Billing Date'] = pd.to_datetime(df_std_copy['Billing Date'], errors = 'coerce')
df_std_copy['Merge Name'] = df_std_copy['Material Number'] + " " + df_std_copy['Billing Date'].dt.strftime('%Y-%m')

In [64]:
df_std = df1.copy()
df_std = df_std[['Material Number', 'Billing Quantity in Base Unit' , 'Billing Date']]
df_std.set_index('Billing Date', inplace = True)
df_std = df_std.reset_index()

In [65]:
# Creating this DataFrame to have 24 months slots for every part number to calculate Standard Deviation & Avergae

# Create a date range for the past 24 months up to December 2024
end_date = pd.to_datetime(current_date)
start_date = end_date - pd.DateOffset(months=23)
date_range = pd.date_range(start=start_date, end=end_date, freq='MS')

# Create a DataFrame with all combinations of 'Manufacturing Material Number' and the date range
part_numbers = df_std_copy['Material Number'].unique()
all_combinations = pd.MultiIndex.from_product([part_numbers, date_range], names=['Material Number', 'Billing Date'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

all_combinations_df['Merge Name'] = all_combinations_df['Material Number'] + " " + all_combinations_df['Billing Date'].dt.strftime('%Y-%m')

In [66]:
# Creating this DataFrame to group billing quantity for each part number based on month

# Convert 'Billing Date' to datetime format
df_std['Billing Date'] = pd.to_datetime(df_std['Billing Date'], errors='coerce')

# Convert 'Billing Quantity in Base Unit' to numeric
df_std['Billing Quantity in Base Unit'] = pd.to_numeric(df_std['Billing Quantity in Base Unit'], errors='coerce')
monthly_demand = df_std.groupby(['Material Number', pd.Grouper(key='Billing Date', freq='M')])['Billing Quantity in Base Unit'].sum().fillna(0).reset_index()

monthly_demand['Merge Name'] = monthly_demand['Material Number'] + " " + monthly_demand['Billing Date'].dt.strftime('%Y-%m')
monthly_demand = monthly_demand.reindex(columns=['Material Number', 'Billing Date', 'Merge Name', 'Billing Quantity in Base Unit'])

C:\Users\610168286\AppData\Local\Temp\ipykernel_21056\4284634732.py:8: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_demand = df_std.groupby(['Material Number', pd.Grouper(key='Billing Date', freq='M')])['Billing Quantity in Base Unit'].sum().fillna(0).reset_index()


In [67]:
df_demand = pd.merge(all_combinations_df, monthly_demand[['Merge Name', 'Billing Quantity in Base Unit']],
                    on = 'Merge Name',
                    how = 'left')

df_demand['Billing Quantity in Base Unit'] = df_demand['Billing Quantity in Base Unit'].fillna(0)

In [68]:
std_dev_per_part = df_demand.groupby('Material Number')['Billing Quantity in Base Unit'].std()
avg_per_part = df_demand.groupby('Material Number')['Billing Quantity in Base Unit'].mean()


Standard_Deviation = pd.DataFrame(std_dev_per_part).reset_index()
Average_Usage = pd.DataFrame(avg_per_part).reset_index()

Standard_Deviation = Standard_Deviation.rename(columns={'Billing Quantity in Base Unit' : 'Standard Deviation'})
Average_Usage = Average_Usage.rename(columns={'Billing Quantity in Base Unit' : 'Average Usage'})

In [69]:
df_group_concentration = pd.merge(df_group_concentration, Average_Usage[['Material Number', 'Average Usage']],
                                            on = 'Material Number',
                                            how = 'left')

df_group_concentration = pd.merge(df_group_concentration, Standard_Deviation[['Material Number', 'Standard Deviation']],
                                            on = 'Material Number',
                                            how = 'left')

## Safety Stock Calculation

In [70]:
# Formula for Safety stock calculation ---> Zfactor * standard deviation * sqrt(Average demand over lead time)

service_level = 0.95
z_score = norm.ppf(service_level).round(2)

# Function to calculate safety stock
def calculate_safety_stock(row, z_score, lead_time_col, std_dev_col):
    return z_score * row[std_dev_col] * np.sqrt(row[lead_time_col]/30)

warehouse_name = warehouse_to_consider

# Function to calculate safety stock for a given DataFrame
def calculate_safety_stock_for_df(df):
    for index, row in df.iterrows():
       for i in warehouse_name:
           lead_time_col = f'{i} Lead Time'
           std_dev_col = 'Standard Deviation'
           safety_stock_col = f'{i} Safety Stock'
           safety_stock = calculate_safety_stock(row, z_score, lead_time_col, std_dev_col)
           
           df.at[index, safety_stock_col] = safety_stock
    return df
    

# Calculate safety stock for both DataFrames
df_group_concentration = calculate_safety_stock_for_df(df_group_concentration)

## Average Demand over leadtime

In [71]:
# Formula for Average Demand over lead time ---> lead time (months) * average

def calculate_average_demand(row, average, leadtime):
    return row[average] * (row[leadtime] / 30)

warehouse_name = warehouse_to_consider


# Function to calculate average demand for a given DataFrame
def calculate_average_demand_for_df(df):
    for index, row in df.iterrows():
        for i in warehouse_to_consider:
            average = 'Average Usage'
            leadtime = f'{i} Lead Time'
            average_demand = f'{i} Average'
            
            # Calculate average demand for the current row and warehouse
            avg_demand = calculate_average_demand(row, average, leadtime)
            
            # Assign the calculated average demand to the DataFrame
            df.at[index, average_demand] = avg_demand
    return df

# Calculate avergae demand over lead time for both DataFrames
df_group_concentration = calculate_average_demand_for_df(df_group_concentration)

## Reorder Point

In [72]:
# Formula for Average Demand over lead time ---> lead time (months) * average

def calculate_reorder_point(row, safety_stock, average):
    return row[safety_stock] + row[average]

warehouse_name = warehouse_to_consider

# Function to calculate reorder point for a given DataFrame
def calculate_reorder_point_for_df(df):
    for index, row in df.iterrows():
        for i in warehouse_to_consider:
            average = f'{i} Average'
            safety_stock = f'{i} Safety Stock'
            reorder_point = f'{i} ROP'
            
            # Calculate reorder point for the current row and warehouse
            rop = calculate_reorder_point(row, safety_stock, average)
            
            # Assign the calculated reorder point to the DataFrame
            df.at[index, reorder_point] = round(rop)
    return df

# Calculate reorder point for both DataFrames
df_group_concentration = calculate_reorder_point_for_df(df_group_concentration)

In [73]:
column_to_drop = ['Standard Deviation', 'Average Usage']

for warehouse in warehouse_to_consider:
    column_to_drop.append(f'{warehouse} Safety Stock')
    column_to_drop.append(f'{warehouse} Average')
    
df_final_group = df_group_concentration.drop(columns=column_to_drop)

In [74]:
#df_demand.to_excel("Not to use - demand try.xlsx")
#df_final_within_900.to_csv('Data_For_Julia_within_900.csv', index = False)
#df_final_commom.to_csv('Data_For_Julia_common.csv', index = False)
#df_merge_group.to_csv('Not to use - Data for Julia.csv', index = False)
#df_group.to_excel('Not to use - df_group.xlsx', index = False)
#df_concentration.to_excel('Final GUA Produced parts warehouse.xlsx', index = False)
df_final_group.to_csv('Data_For_Julia_10.csv', index = False)